In [1]:
import pandas as pd
import numpy as np
from random import random
import json
import math

# Mapping and Visuals
import geog
import geopandas as gpd
import shapely
from shapely.geometry import Point, Polygon, mapping

In [2]:
# POSSIBLE LOCATIONS TO USE FOR MAPPING, WITH COORDINATES
coords_dict = {
    "Paradise": (39.76, -121.62),
    "Campbellville": (40.02, -121.723),
    "Chico": (39.728, -121.837),
    "Junction House": (39.316, -120.81),
    "Dodgeland": (39.544, -121.906)
}

loc_dict = {
    "City": [
        "Paradise",
        "Cambellville",
        "Chico",
        "Junction House",
        "Dodgeland"
    ],
    "Coords": list(coords_dict.values())
}

df = pd.DataFrame(loc_dict)

df["long"] = [i[1] for i in df["Coords"]]
df["lat"] = [i[0] for i in df["Coords"]]
df.head()

# Reference to DSI NYC III Cohort

,City,Coords,long,lat
0,Paradise,"(39.76, -121.62)",-121.620,39.760
1,Cambellville,"(40.02, -121.723)",-121.723,40.020
2,Chico,"(39.728, -121.837)",-121.837,39.728
3,Junction House,"(39.316, -120.81)",-120.810,39.316
4,Dodgeland,"(39.544, -121.906)",-121.906,39.544


In [3]:
def rand_cluster(n,c,r): 
    # N = NUMBER COORDS TO GENERATE
    # C = "CENTER" AT WHICH EVERYTHING WILL CLUSTER
    # R = "RADIUS" 
    x,y = c 
    points = []
    for i in range(n):
        theta = 2 * math.pi * random()
        s = r * random()
        points.append((x+s*math.cos(theta), y+s*math.sin(theta)))
    return points
# https://stackoverflow.com/questions/44356063/how-to-generate-a-set-of-random-points-within-a-given-x-y-coordinates-in-an-x

### Non-Urgent Tweets

In [4]:
non_urgent = pd.read_csv("../projects/p5/iter1/Client-Project/datasets/df_nonurgent.csv") # READ IN
non_urgent.drop(columns=["Unnamed: 0"], inplace=True) # DROP UNNEEDED COLS

# CHECK OUTPUT
print(non_urgent.shape)
non_urgent.head()

(4133, 6)


,cleaned_message,urgent_cos_sim,non_urgent_cos_sim,urgent,text,username
0,smokea smoke investigation outside structure n...,0.158059,0.164721,0,SMOKEA - SMOKE INVESTIGATION OUTSIDE A STRUCTU...,Mult Co Fire/EMS log
1,med medical se nd ave se morrison st morrison ...,0.141777,0.144964,0,MED - MEDICAL at SE 2ND AVE / SE MORRISON ST &...,Mult Co Fire/EMS log
2,smokea smoke investigation outside structure s...,0.149254,0.155813,0,SMOKEA - SMOKE INVESTIGATION OUTSIDE A STRUCTU...,Mult Co Fire/EMS log
3,buy two get one free decorate tree ghoulish de...,0.142310,0.147313,0,"Buy two get one free, decorate your tree with ...",imafoolishmortal
4,bay area thankful rain clean air fire brutal n...,0.142018,0.143126,0,The Bay Area is so thankful for rain and clean...,Justine Reiss PRmama


In [7]:
# add coords to non_urgent
non_urgent_coords = rand_cluster(non_urgent.shape[0], df["Coords"][0], .22)
non_urgent_coords = pd.Series(non_urgent_coords)
non_urgent["coordinates"] = non_urgent_coords
non_urgent[["lat", "long"]] = pd.DataFrame(non_urgent["coordinates"].tolist(), index=non_urgent.index)
non_urgent.tail()

,cleaned_message,urgent_cos_sim,non_urgent_cos_sim,urgent,text,username,coordinates,lat,long
4128,rockcreek fire unofficial new fire report deta...,0.140829,0.141298,0,#RockCreek2Fire unofficial new fire report; de...,Wildfire Bot (WA/OR)-Unofficial 🔥🤖,"(39.655197584365204, -121.80227540990977)",39.655198,-121.802275
4129,smoke distant fire san jose california,0.144814,0.145865,0,"Smoke from a distant fire... @ San Jose, Calif...",Scott Wendler,"(39.91386842153125, -121.6861438849976)",39.913868,-121.686144
4130,buttonfire unofficial new fire report detail u...,0.142144,0.146126,0,#ButtonFire unofficial new fire report; detail...,Wildfire Bot (WA/OR)-Unofficial 🔥🤖,"(39.77059975256987, -121.63631614182962)",39.770600,-121.636316
4131,fire haze sonoma right fairmont sonoma mission...,0.141530,0.146908,0,"Fire haze in Sonoma right now, too. @ Fairmont...",D.J. Grothe,"(39.82906612529524, -121.64746319709874)",39.829066,-121.647463
4132,help live paradise call text help pacific real...,0.141037,0.147459,0,"I can help YOU live here in Paradise too, jus...",Jane Loveday,"(39.85478388278124, -121.65931830954312)",39.854784,-121.659318


In [321]:
non_urgent = gpd.GeoDataFrame(non_urgent,
                              geometry=gpd.points_from_xy(non_urgent["long"], non_urgent["lat"])) 
non_urgent.head()

,cleaned_message,urgent_cos_sim,non_urgent_cos_sim,urgent,text,username,coordinates,lat,long,geometry
0,smokea smoke investigation outside structure n...,0.158059,0.164721,0,SMOKEA - SMOKE INVESTIGATION OUTSIDE A STRUCTU...,Mult Co Fire/EMS log,"(39.69968802264566, -121.44489472107708)",39.699688,-121.444895,POINT (-121.44489 39.69969)
1,med medical se nd ave se morrison st morrison ...,0.141777,0.144964,0,MED - MEDICAL at SE 2ND AVE / SE MORRISON ST &...,Mult Co Fire/EMS log,"(39.71143585846352, -121.52301797778696)",39.711436,-121.523018,POINT (-121.52302 39.71144)
2,smokea smoke investigation outside structure s...,0.149254,0.155813,0,SMOKEA - SMOKE INVESTIGATION OUTSIDE A STRUCTU...,Mult Co Fire/EMS log,"(39.84973066506726, -121.78415537576122)",39.849731,-121.784155,POINT (-121.78416 39.84973)
3,buy two get one free decorate tree ghoulish de...,0.142310,0.147313,0,"Buy two get one free, decorate your tree with ...",imafoolishmortal,"(39.660182881421626, -121.44326546534647)",39.660183,-121.443265,POINT (-121.44327 39.66018)
4,bay area thankful rain clean air fire brutal n...,0.142018,0.143126,0,The Bay Area is so thankful for rain and clean...,Justine Reiss PRmama,"(39.82131727895328, -121.70039247305644)",39.821317,-121.700392,POINT (-121.70039 39.82132)


### Urgent Tweets

In [5]:
urgent = pd.read_csv("../projects/p5/iter1/Client-Project/datasets/df_urgent.csv") # READ IN
urgent.drop(columns=["Unnamed: 0"], inplace=True) # DROP UNNEEDED COLS

# CHECK OUTPUT
print(urgent.shape)
urgent.head()

(7651, 6)


,cleaned_message,urgent_cos_sim,non_urgent_cos_sim,urgent,text,username
0,med medical block ne th ave portland portland ...,0.148701,0.137452,1,"MED - MEDICAL at 700 BLOCK OF NE 78TH AVE, POR...",Mult Co Fire/EMS log
1,med medical se nd ave se johnson creek blvd po...,0.149998,0.135317,1,MED - MEDICAL at SE 32ND AVE / SE JOHNSON CREE...,Mult Co Fire/EMS log
2,valley fire valley fire state park,0.144539,0.141861,1,Valley of Fire @ Valley of Fire State Park htt...,MichelleBot
3,med medical block ne rodney ave portland portl...,0.145074,0.131240,1,"MED - MEDICAL at 2000 BLOCK OF NE RODNEY AVE, ...",Mult Co Fire/EMS log
4,special shout llyk comin fire video downtown l...,0.144783,0.133166,1,Special shout out to @m0llyk4y for comin throu...,Houdini


In [328]:
# add coors urgent
urgent_coords = rand_cluster(urgent.shape[0], df["Coords"][0], .22)
urgent_coords = pd.Series(urgent_coords)
urgent["coordinates"] = urgent_coords
urgent[["lat", "long"]] = pd.DataFrame(urgent["coordinates"].tolist(), index=urgent.index)
urgent.tail()

,cleaned_message,urgent_cos_sim,non_urgent_cos_sim,urgent,text,username,coordinates,lat,long
7646,almcom monitored commercial fire alarm block s...,0.136933,0.135360,1,ALMCOM - MONITORED COMMERCIAL FIRE ALARM at 14...,Mult Co Fire/EMS log,"(39.78427057540957, -121.5204217164063)",39.784271,-121.520422
7647,vega paradise nevada,0.148155,0.126041,1,"Vegasss! @ Paradise, Nevada https://www.instag...",Wai Reyes,"(39.705952040949654, -121.62010055446402)",39.705952,-121.620101
7648,sun appears like pink blob due smoke camp fire...,0.149713,0.141497,1,Sun appears like a pink blob due to smoke from...,Jonathan,"(39.7176560302525, -121.72027641199409)",39.717656,-121.720276
7649,pray calabasas thousand oak fire serious deadl...,0.145576,0.141711,1,Pray for Calabasas \nand Thousand Oaks \nThis ...,Scott Sanders,"(39.694707334619885, -121.57219129007774)",39.694707,-121.572191
7650,east bay area smoke ha turned sun red bayarea ...,0.144723,0.143298,1,East Bay Area Smoke has turned the Sun red #...,John Meredith,"(39.87304728904356, -121.51476876940242)",39.873047,-121.514769


In [329]:
urgent = gpd.GeoDataFrame(urgent,
                          geometry=gpd.points_from_xy(urgent["long"], urgent["lat"])) 
urgent.head()

,cleaned_message,urgent_cos_sim,non_urgent_cos_sim,urgent,text,username,coordinates,lat,long,geometry
0,med medical block ne th ave portland portland ...,0.148701,0.137452,1,"MED - MEDICAL at 700 BLOCK OF NE 78TH AVE, POR...",Mult Co Fire/EMS log,"(39.62438891197958, -121.76761859120393)",39.624389,-121.767619,POINT (-121.76762 39.62439)
1,med medical se nd ave se johnson creek blvd po...,0.149998,0.135317,1,MED - MEDICAL at SE 32ND AVE / SE JOHNSON CREE...,Mult Co Fire/EMS log,"(39.757332371260134, -121.6188134634258)",39.757332,-121.618813,POINT (-121.61881 39.75733)
2,valley fire valley fire state park,0.144539,0.141861,1,Valley of Fire @ Valley of Fire State Park htt...,MichelleBot,"(39.818885240150564, -121.66413093279144)",39.818885,-121.664131,POINT (-121.66413 39.81889)
3,med medical block ne rodney ave portland portl...,0.145074,0.131240,1,"MED - MEDICAL at 2000 BLOCK OF NE RODNEY AVE, ...",Mult Co Fire/EMS log,"(39.69547544590889, -121.60151890300534)",39.695475,-121.601519,POINT (-121.60152 39.69548)
4,special shout llyk comin fire video downtown l...,0.144783,0.133166,1,Special shout out to @m0llyk4y for comin throu...,Houdini,"(39.78546141323192, -121.81032331259583)",39.785461,-121.810323,POINT (-121.81032 39.78546)


### Save New DFs to File

In [330]:
# save the 2 new df's
non_urgent.to_csv("./datasets/non_urgent_with_coords.csv")
urgent.to_csv("./datasets/urgent_with_coords.csv")